In [32]:
import sympy as smp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from itertools import product

In [33]:
smp.init_printing(use_unicode=True)

In [34]:
class FileSystemManager:
    def __init__(self):
        pass
    def open_file(self, text, filepath, flag):
        """
        Opens a file in read or write mode.
        
        :param text: Text to write to the file if flag is 'w'
        :param filepath: Path of the file to open
        :param flag: 'r' to read, 'w' to write
        :return: File content if reading, None if writing
        """
        try:
            if flag == 'r':
                with open(filepath, 'r') as f:
                    data = f.read()
                    return data  # return instead of print
            elif flag == 'w':
                with open(filepath, 'w') as f:
                    f.write(text)
            else:
                raise ValueError("Unsupported flag. Use 'r' for read or 'w' for write.")
        except FileNotFoundError:
            print(f"Error: File {filepath} not found.")
        except IOError as e:
            print(f"Error accessing file {filepath}: {e}")

    def latex(self):
        pass

In [35]:
class MagmaCalculator:
    """
    A class to interact with the Magma Calculator webpage and submit code for evaluation.
    
    Attributes:
    ----------
    output_file : str
        The name of the file where the output from the Magma Calculator will be saved.
    driver : webdriver.Chrome
        A Chrome web driver instance to automate browser interaction.
    url : str
        The URL of the Magma Calculator page.
    
    Methods:
    -------
    submit_code(code):
        Submits the given Magma code to the calculator and saves the result to a file.
    
    close():
        Closes the browser session.
    """
    
    def __init__(self, output_file="MagmaCalcResult"):
        """
        Initializes the MagmaCalculator with the given output file name.
        
        Parameters:
        ----------
        output_file : str, optional
            The name of the file where the result will be saved (default is "output.txt").
        """
        self.url = "http://magma.maths.usyd.edu.au/calc/"
        self.output_file = output_file
        self.file_manager = FileSystemManager()

    def submit_code(self, code):
        """
        Submits the given code to the Magma Calculator and saves the result to the output file.
        
        Parameters:
        ----------
        code : str
            The Magma code to be submitted for evaluation.
        
        Actions:
        -------
        - Opens the Magma Calculator webpage.
        - Finds the input box and enters the code.
        - Clicks the submit button.
        - Waits for the result to load.
        - Retrieves the result and writes it to the specified output file.
        """
        driver = webdriver.Chrome()
        try:
            driver.get(self.url)
            
            input_box = driver.find_element(By.ID, "input")
            input_box.clear()
            input_box.send_keys(code)
            
            submit_button = driver.find_element(By.XPATH, "//input[@value='Submit']")
            submit_button.click()
            
            time.sleep(5)
            
            result_element = driver.find_element(By.ID, "result")
            result_text = result_element.get_attribute('value')
            
            self.file_manager.open_file(result_text, self.output_file, 'w')
        finally:
            driver.quit()

In [36]:
class FlynnLeprevost:
    def __init__(self, m=11, g=2, inf_count=1, range_limit=15, curves_path="foundCurves.txt", magma_result_path="MagmaCalcResult"):
        """"""
        self.x, self.y = smp.symbols('x y')
        self.f = smp.symbols('f')
        self.V1, self.V2, self.W1, self.W2 = smp.symbols('V1 V2 W1 W2') 
        self.w = smp.symbols('w')
        self.m = m
        self.g = g
        self.range_limit = range_limit
        if inf_count == 1:
            self.deg = 2 * g + 1
        elif inf_count == 2:
            self.deg = 2 * g + 2
        else:
            raise ValueError("inf_count must be 1 or 2")
        self.constraints = [self.deg, self.deg + 2*self.g]
        self.coeffs = self._get_coeffs()
        self.curves_path = curves_path
        self.magma_calc = MagmaCalculator(magma_result_path)
        self.file_manager = FileSystemManager()

    def factor_coeffs(self, f, val):
        """
        Factors the coefficients of a polynomial and computes the sum of 
        each factored coefficient multiplied by a power of the given value.
        
        :param f: The polynomial expression whose coefficients will be factored.
        :param val: The symbol or value to use as the base for powers in the sum.
        
        :return: The sum of the factored coefficients of the polynomial `f`,
                each multiplied by the corresponding power of `val`.
                
        Example:
        If f = 2*x**2 + 3*x + 4 and val = x, the function will factor each 
        coefficient (if possible) and return the sum 4 + 3*x + 2*x**2.
        """
        return sum(smp.factor(coeff)*val**i for i, coeff in enumerate(reversed(smp.Poly(f, val).all_coeffs())))

    def torsion_point_order_divisor(self, a1, b1, a2, b2):
        """Calculate the divisor of torsion point order using the determinant."""
        matrix = smp.Matrix([[a1, b1], [a2, b2]])
        m = abs(matrix.det())
        return m
    
    def check_order(self, equation):
        """Function that helps to check an order of possible torison points of given function"""
        code = f"""
        P<x> := PolynomialRing(Rationals());
        C1 := HyperellipticCurve({equation});
        J1 := Jacobian(C1);
        TorsionSubgroup(J1);
        ClebschInvariants(C1);
        IgusaClebschInvariants(C1);
        IgusaInvariants(C1);"""
        self.magma_calc.submit_code(code, self.magma_calc.output_file)

    def divisor_deg(self, deg_V_j, deg_W_j):
        """
        Checks the divisor degree of given function
        deg*: degree of divisor
        deg: degree  of polynomial 
        deg* u_j = max(2 deg(V_j), 2 deg(W_j) + 2g + 1)
        """
        return max(2* smp.degree(deg_V_j, self.x), 2 * smp.degree(deg_W_j, self.x) + self.deg)

    def in_constraints(self, coeffs_set):
        return (abs(coeffs_set[0]) + abs(coeffs_set[1]) <= abs(self.constraints[1])) and (abs(coeffs_set[0]) + abs(coeffs_set[1]) >= abs(self.constraints[0])) and (abs(coeffs_set[2]) + abs(coeffs_set[3]) <= abs(self.constraints[1])) and (abs(coeffs_set[2]) + abs(coeffs_set[3]) >= abs(self.constraints[0]))
    
    def coeffs_restrictions(self, coeffs_set):
        return self.torsion_point_order_divisor(*coeffs_set) == self.m and self.in_constraints(coeffs_set)

    # def get_coeffs(self):
    #     values = range(1, self.range_limit)
    #     final_coeffs = []

    #     coeffs_sets = product(values, repeat=4)

    #     for coeffs_set in coeffs_sets:
    #         if self.coeffs_restrictions(coeffs_set):
    #             final_coeffs.append(coeffs_set)

    #     return final_coeffs
    
    def _get_coeffs(self):
        values = [i for i in range(1, self.range_limit)]
        values_last =[i for i in range(-self.range_limit,0)] 
        final_coeffs = []
        coeffs_sets = [[a1, b1, a2, b2] for a1 in values for b1 in values for a2 in values for b2 in values]
        for coeffs_set in coeffs_sets:
            if self.coeffs_restrictions(coeffs_set):
                final_coeffs.append(coeffs_set)

        return final_coeffs
    
    def get_system_information(self, coeffs_set):  
        a1,b1,a2,b2 = abs(coeffs_set[0]),abs(coeffs_set[1]),abs(coeffs_set[2]),abs(coeffs_set[3])

        deg_V1 = int(np.floor((a1 + b1) / 2))
        deg_V2 = int(np.floor((a2 + b2) / 2))
        deg_W1 = int(np.floor((a1 + b1 - self.deg) / 2))
        deg_W2 = int(np.floor((a2 + b2 - self.deg) / 2))

        alpha, beta = min(a1, a2), min(b1, b2)
        gamma = smp.symbols('gamma')
        a = smp.symbols('a')

        cum_deg_left = 2 * max(deg_V1 + deg_W2, deg_V2 + deg_W1)
        cum_deg_right = alpha + beta + max(2*deg_W2 + (a1 - alpha) + (b1 - beta), 2*deg_W1 + (a2 - alpha) + (b2 - beta))
        
        deg_w = int(cum_deg_left/2 - alpha)

        if (a1 + b1) % 2 == 0 and (a2 + b2) % 2 == 0: 
            lc_V1 = 1
            lc_V2 = 1
            lc_W1 = 1
            lc_W2 = a
            lc_f = -gamma
        elif (a1 + b1) % 2 == 0 and (a2 + b2) % 2 != 0:
            lc_V1 = 1
            lc_V2 = a
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 != 0:
            lc_V1 = 1
            lc_V2 = a
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 == 0:
            lc_V1 = a
            lc_V2 = 1    
            lc_W1 = 1
            lc_W2 = 1
            lc_f = -gamma
            
        system = {
            'coeffs_set': coeffs_set,
            'deg(V1)': deg_V1,
            'deg(V2)': deg_V2,
            'deg(W1)': deg_W1,
            'deg(W2)': deg_W2,  
            'deg(w)' : deg_w,
            'cum_deg_left': cum_deg_left,
            'cum_deg_right': cum_deg_right, 
            'lc(W1)': lc_W1,
            'lc(W2)': lc_W2,
            'lc(V1)': lc_V1,
            'lc(V2)': lc_V2,
            'lc(f)': lc_f,
        }
        return system

    def get_systems_information(self):
        coeffs_sets = self.coeffs
        systems_information = []
        for coeffs_set in coeffs_sets:
            systems_information.append(self.get_system_information(coeffs_set))

        return systems_information
    
    def get_system(self, coeffs_set):
        system_information = self.get_system_information(coeffs_set)

        left_part_1 = self.V1**2 - self.W1**2 * self.f
        left_part_2 = self.V2**2 - self.W2**2 * self.f

        gamma_1 = smp.symbols('gamma_1')  
        gamma_2 = smp.symbols('gamma_2')
        power_x = smp.symbols('n')     
        power_x_minus_1 = smp.symbols('m')  

        equation_1 = gamma_1 * self.x**power_x * (self.x - 1)**power_x_minus_1 
        equation_2 = gamma_2 * self.x**power_x * (self.x - 1)**power_x_minus_1 

        right_part_1 = equation_1.subs({power_x: abs(coeffs_set[0]), power_x_minus_1: abs(coeffs_set[1])})
        right_part_2 = equation_2.subs({power_x: abs(coeffs_set[2]), power_x_minus_1: abs(coeffs_set[3])})

        eq_1 = smp.Eq(left_part_1, right_part_1)
        eq_2 = smp.Eq(left_part_2, right_part_2)

        system = {
            'coeffs_set': system_information ['coeffs_set'],
            'deg(V1)': system_information ['deg(V1)'],
            'deg(V2)': system_information ['deg(V2)'],
            'deg(W1)': system_information ['deg(W1)'],
            'deg(W2)': system_information ['deg(W2)'],
            'cum_deg_left': system_information ['cum_deg_left'],
            'cum_deg_right': system_information ['cum_deg_right'],
            'eq_1': eq_1,
            'eq_2': eq_2,
        }
        return system

    def get_systems(self):
        final_systems = []
        coeffs_set = self.coeffs
        for coeffs in coeffs_set:
            system = self.get_system(coeffs)
            final_systems.append(system)

        return final_systems

    def get_equation(self, coeffs_set):
        system_information = self.get_system_information(coeffs_set)

        gamma_1 = smp.symbols('gamma')  
        gamma_2 = smp.symbols('gamma') 

        a1,b1,a2,b2 = abs(coeffs_set[0]),abs(coeffs_set[1]),abs(coeffs_set[2]),abs(coeffs_set[3])

        alpha, beta = min(a1, a2), min(b1, b2)

        if (a1 + b1) % 2 == 0 and (a2 + b2) % 2 == 0: 
            gamma_1 = 1
            gamma_2 = 1
        elif (a1 + b1) % 2 == 0 and (a2 + b2) % 2 != 0:
            gamma_1 = 1
        elif (a1 + b1) % 2 != 0 and (a2 + b2) % 2 == 0:
            gamma_2 = 1

        left_part = (self.V1 * self.W2 - self.V2 * self.W1)*(self.V1 * self.W2 + self.V2 * self.W1)
        right_part = self.x**alpha * (self.x - 1)**beta * (gamma_1 * self.W2**2 * self.x**(a1 - alpha) * (self.x - 1)**(b1 - beta)
                                                         - gamma_2 * self.W1**2 * self.x**(a2 - alpha) * (self.x - 1)**(b2 - beta))

        equation = smp.Eq(left_part, right_part)

        equation = {
            'coeffs_set': system_information ['coeffs_set'],
            'deg(V1)': system_information ['deg(V1)'],
            'deg(V2)': system_information ['deg(V2)'],
            'deg(W1)': system_information ['deg(W1)'],
            'deg(W2)': system_information ['deg(W2)'],
            'deg(w)' : system_information ['deg(w)'], 
            'cum_deg_left': system_information ['cum_deg_left'],
            'cum_deg_right': system_information ['cum_deg_right'],
            'equation': equation,
            'lc(W1)': system_information ['lc(W1)'],
            'lc(W2)': system_information ['lc(W2)'],
            'lc(V1)': system_information ['lc(V1)'],
            'lc(V2)': system_information ['lc(V2)'],
            'lc(f)': system_information ['lc(f)'],
        }
        return equation

    def get_equations(self):
        simpyified_equations = []
        for coeffs_set in self.coeffs:
            simpyified_equations.append(self.get_equation(coeffs_set))
        return simpyified_equations
    
    def get_system_fe(self, coeffs_set):
        pass 

    def get_systems_fe(self):
        systems = []
        for coeffs_set in self.coeffs:
            systems.append(self.get_system_fe(coeffs_set))
        return systems

In [37]:
model = FlynnLeprevost()
model.coeffs

[[1, 4, 4, 5], [1, 5, 3, 4], [1, 7, 2, 3], [1, 8, 2, 5], [2, 3, 1, 7], [2, 3,  ↪

↪ 5, 2], [2, 5, 1, 8], [2, 5, 3, 2], [2, 7, 3, 5], [3, 2, 2, 5], [3, 2, 7, 1], ↪

↪  [3, 4, 1, 5], [3, 4, 5, 3], [3, 5, 2, 7], [3, 5, 4, 3], [4, 1, 5, 4], [4, 3 ↪

↪ , 3, 5], [4, 3, 5, 1], [4, 5, 1, 4], [5, 1, 4, 3], [5, 2, 2, 3], [5, 2, 8, 1 ↪

↪ ], [5, 3, 3, 4], [5, 3, 7, 2], [5, 4, 4, 1], [7, 1, 3, 2], [7, 2, 5, 3], [8, ↪

↪  1, 5, 2]]

In [38]:
systems_information = model.get_systems_information()
systems = model.get_systems()
simpyified_equations = model.get_equations()

In [39]:
cur_sys = systems[-2]

In [40]:
cur_sys['eq_1']

  2     2         7        2
V₁  - W₁ ⋅f = γ₁⋅x ⋅(x - 1) 

In [41]:
cur_sys['eq_2']

  2     2         5        3
V₂  - W₂ ⋅f = γ₂⋅x ⋅(x - 1) 

In [42]:
simpyified_equations

[{'coeffs_set': [1, 4, 4, 5],
  'deg(V1)': 2,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**4*(-W1**2*gamma*x**3*(x - 1) + W2**2*gamma)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 5, 3, 4],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 0,
  'deg(W2)': 1,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**4*(-W1**2*gamma*x**2 + W2**2*(x - 1))),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [1, 7, 2, 3],
  'deg(V1)': 4,
  'deg(V2)': 2,
  'deg(W1)': 1,
  'deg(W2)': 0,
  'deg(w)': 3,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x*(x - 1)**3*(-W1**2*gamma*x + W2**2*(x - 1)**4)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gam

In [43]:
def filter_by_w(deg_w, simpyified_equations):
    filtered_sys = []
    for sy in simpyified_equations:
        if sy['deg(w)'] == deg_w:
            filtered_sys.append(sy)
    return filtered_sys

In [44]:
def filter_by_w(deg_w, simpyified_equations):
    filtered_sys = []
    for sy in simpyified_equations:
        if sy['deg(w)'] == deg_w:
            filtered_sys.append(sy)
    return filtered_sys

In [45]:
filtered = filter_by_w(1, simpyified_equations)

In [59]:
filtered[-1]['equation']

                                   5         ⎛    2               2    3⎞
(V₁⋅W₂ - V₂⋅W₁)⋅(V₁⋅W₂ + V₂⋅W₁) = x ⋅(x - 1)⋅⎝- W₁ ⋅γ⋅(x - 1) + W₂ ⋅γ⋅x ⎠

In [47]:
filtered = filter_by_w(0, simpyified_equations)

In [48]:
len(filtered)

6

In [49]:
filtered

[{'coeffs_set': [4, 1, 5, 4],
  'deg(V1)': 2,
  'deg(V2)': 4,
  'deg(W1)': 0,
  'deg(W2)': 2,
  'deg(w)': 0,
  'cum_deg_left': 8,
  'cum_deg_right': 9,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**4*(x - 1)*(-W1**2*gamma*x*(x - 1)**3 + W2**2*gamma)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gamma},
 {'coeffs_set': [4, 3, 5, 1],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 1,
  'deg(W2)': 0,
  'deg(w)': 0,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**4*(x - 1)*(-W1**2*x + W2**2*gamma*(x - 1)**2)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': a,
  'lc(V2)': 1,
  'lc(f)': -gamma},
 {'coeffs_set': [5, 1, 4, 3],
  'deg(V1)': 3,
  'deg(V2)': 3,
  'deg(W1)': 0,
  'deg(W2)': 1,
  'deg(w)': 0,
  'cum_deg_left': 8,
  'cum_deg_right': 8,
  'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**4*(x - 1)*(-W1**2*gamma*(x - 1)**2 + W2**2*x)),
  'lc(W1)': 1,
  'lc(W2)': 1,
  'lc(V1)': 1,
  'lc(V2)': a,
  'lc(f)': -gam

In [50]:
eq_1 = filtered[1]

In [51]:
eq_1 

{'coeffs_set': [4, 3, 5, 1],
 'deg(V1)': 3,
 'deg(V2)': 3,
 'deg(W1)': 1,
 'deg(W2)': 0,
 'deg(w)': 0,
 'cum_deg_left': 8,
 'cum_deg_right': 8,
 'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**4*(x - 1)*(-W1**2*x + W2**2*gamma*(x - 1)**2)),
 'lc(W1)': 1,
 'lc(W2)': 1,
 'lc(V1)': a,
 'lc(V2)': 1,
 'lc(f)': -gamma}

In [52]:
eq_1['equation']

                                   4         ⎛    2       2          2⎞
(V₁⋅W₂ - V₂⋅W₁)⋅(V₁⋅W₂ + V₂⋅W₁) = x ⋅(x - 1)⋅⎝- W₁ ⋅x + W₂ ⋅γ⋅(x - 1) ⎠

In [53]:
coeff = [2,3,5,2]
system = model.get_system(coeff)
simpyified_equation = model.get_equation(coeff)

In [54]:
system

{'coeffs_set': [2, 3, 5, 2],
 'deg(V1)': 2,
 'deg(V2)': 3,
 'deg(W1)': 0,
 'deg(W2)': 1,
 'cum_deg_left': 6,
 'cum_deg_right': 7,
 'eq_1': Eq(V1**2 - W1**2*f, gamma_1*x**2*(x - 1)**3),
 'eq_2': Eq(V2**2 - W2**2*f, gamma_2*x**5*(x - 1)**2)}

In [55]:
system['eq_1']

  2     2         2        3
V₁  - W₁ ⋅f = γ₁⋅x ⋅(x - 1) 

In [56]:
system['eq_2']

  2     2         5        2
V₂  - W₂ ⋅f = γ₂⋅x ⋅(x - 1) 

In [57]:
simpyified_equation

{'coeffs_set': [2, 3, 5, 2],
 'deg(V1)': 2,
 'deg(V2)': 3,
 'deg(W1)': 0,
 'deg(W2)': 1,
 'deg(w)': 1,
 'cum_deg_left': 6,
 'cum_deg_right': 7,
 'equation': Eq((V1*W2 - V2*W1)*(V1*W2 + V2*W1), x**2*(x - 1)**2*(-W1**2*gamma*x**3 + W2**2*gamma*(x - 1))),
 'lc(W1)': 1,
 'lc(W2)': 1,
 'lc(V1)': 1,
 'lc(V2)': a,
 'lc(f)': -gamma}

In [58]:
simpyified_equation['equation']

                                   2        2 ⎛    2    3     2          ⎞
(V₁⋅W₂ - V₂⋅W₁)⋅(V₁⋅W₂ + V₂⋅W₁) = x ⋅(x - 1) ⋅⎝- W₁ ⋅γ⋅x  + W₂ ⋅γ⋅(x - 1)⎠